# **Mounting Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Importing Libraries**

In [ ]:
pip install pmdarima

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.1 MB/s eta 0:00:00


In [ ]:
import re
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
from datetime import date
from datetime import time
from datetime import timedelta
from pathlib import Path
from google.cloud import bigquery as bq
from IPython.display import display
import seaborn as sns
import os
import scipy
import pandas_gbq
from google.cloud import bigquery_storage
from google.oauth2 import service_account
from google.colab import files
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import plotly.express as px
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
plt.style.use('seaborn')
from sklearn.preprocessing import MinMaxScaler
from scipy import stats
from prophet import Prophet
from datetime import datetime
from prophet.plot import plot_plotly, plot_components_plotly
import warnings
warnings.filterwarnings("ignore")


from sklearn.linear_model import LinearRegression


# libraries for time series
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
import pmdarima as pm # auto-arima to determine ARIMA order
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error



# **Creating BigQuery Client**

# **Fetching data in weeks for each business**

In [ ]:
# partner = 'Partner X sellers'

query = '''
SELECT business_name, DATE_TRUNC(order_date, WEEK) AS week, SUM(amount) AS sales_amount,
FROM `database_name.schema_name.sales_data_raw`
WHERE amount > 0
      AND order_date <= DATE(current_datetime())
      AND partner = 'partner X sellers'
GROUP BY business_name, week
HAVING count(product) > 0
ORDER by business_name, week
'''

job = client.query(query)
data = job.result()
data = data.to_dataframe()
data['week'] = pd.to_datetime(data['week'])
data

,business_name,week,sales_amount
0,4B Foods - RDS,2022-06-26,99100.0
1,4B Foods - RDS,2022-07-03,137740.0
2,4B Foods - RDS,2022-07-10,75375.0
3,4B Foods - RDS,2022-07-17,119730.0
4,4B Foods - RDS,2022-07-24,138220.0
...,...,...,...
1425,Zoya Azaan Trader,2023-04-09,75048.0
1426,Zoya Azaan Trader,2023-04-16,1800.0
1427,Zoya Azaan Trader,2023-04-23,550.0
1428,Zoya Azaan Trader,2023-04-30,13500.0


In [ ]:
# businesses with 20 or more weeks

businesses_with_10_or_more_weeks = (data
                                    .groupby('business_name')
                                    .agg(weeks = ("week", "count"),
                                         max_sales_amount = ("sales_amount", "max"),
                                         min_sales_amount = ("sales_amount", "min")
                                         )
                                    .reset_index()
                                    .sort_values(by=['business_name', 'weeks'], ascending=[True, True])
                                    .query('weeks >= 20')
                                    # .query('max_sales_amount != min_sales_amount')
                                    )

businesses_with_10_or_more_weeks
businesses_filtered_10_weeks = data.loc[data['business_name'].isin(businesses_with_10_or_more_weeks['business_name']), :]


In [ ]:
len(businesses_filtered_10_weeks['business_name'].unique())

26

# **Using an Automated ARIMA Order Selection Method**

**How does Auto-Arima (auto_arima function) work?**

1. Differencing Check: pmdarima.auto_arima internally checks whether differencing is necessary to make the time series stationary. It looks at the original series and conducts statistical tests (like the Augmented Dickey-Fuller test) to assess stationarity. If the series is already stationary (or if it's seasonal), it may not apply any differencing.

2. Order Selection: The function searches through various combinations of orders (p, d, q) and seasonal orders (P, D, Q, s) to determine the best ARIMA or SARIMA model based on criteria such as AIC or BIC. It considers differencing orders (d and D) as part of the order selection process.

3. Best Model: After the search process, pmdarima.auto_arima selects the best-fitting model, including the appropriate values for d and D if differencing is necessary.

Because of this automated differencing check and order selection process, you can provide the original time series data to pmdarima.auto_arima without pre-differencing it. The function will determine whether differencing is required and, if so, what order of differencing (d) is appropriate.

In most cases, this automated approach simplifies the process of determining the ARIMA orders and ensures that the resulting model is appropriate for the data, whether differencing is needed or not.

**auto_arima function paramters:**


In the pm.auto_arima() function call from the pmdarima library, the parameters control the behavior of the automated ARIMA model order selection process. Here's an explanation of each parameter:

- sales_data: This is the input time series data that you want to model using ARIMA. It should be a one-dimensional array-like object (e.g., a list, NumPy array, or pandas Series) containing the time series values.

- seasonal=False: This parameter specifies whether to consider seasonality when determining the ARIMA orders. Setting it to False means that the automated order selection process will not consider seasonal components. If you have data with clear seasonal patterns, you may set this parameter to True to enable seasonal order selection.

- stepwise=True: When set to True, the stepwise parameter enables a stepwise search algorithm for determining the best ARIMA orders. The stepwise search explores a range of potential orders and selects the one with the lowest AIC (Akaike Information Criterion) score. This is a commonly used approach for automated ARIMA order selection.

- suppress_warnings=True: This parameter controls whether to suppress warnings generated during the automated order selection process. When set to True, it prevents the function from displaying warnings that might occur during the order search. Suppressing warnings can be useful if you prefer a cleaner output.


The pm.auto_arima() function uses a combination of grid search and optimization techniques to determine the optimal ARIMA orders (p, d, q) for a given time series dataset. It searches through various combinations of orders, evaluates their performance using AIC, and selects the orders that minimize AIC as the best-fit ARIMA model.



There are 2 methods for selecting model hyperparameters and optimizing machine learning or time series models. They differ in their search strategies and complexity:

**Grid Search:**
- Search Strategy: In a grid search, you specify a range of possible values for each hyperparameter, and the search method evaluates every possible combination of hyperparameters within those specified ranges. It creates a grid of all possible combinations and systematically tests each one.

- Exhaustive Search: Grid search is exhaustive, meaning it doesn't skip any combination. It tests all possibilities, making it a comprehensive approach.

- Complexity: Grid search can be computationally expensive, especially when dealing with a large number of hyperparameters or a wide range of values for each hyperparameter. The search space grows exponentially with the number of hyperparameters.

- Advantages: Grid search is guaranteed to find the best combination of hyperparameters within the specified ranges, given enough computational resources and time. It is a simple and straightforward approach.

- Disadvantages: It can be slow and computationally intensive, especially when dealing with a large search space. It may not be feasible in situations with limited computational resources.


**Step-Wise Search:**

- Search Strategy: A step-wise search, also known as a sequential search, starts with an initial set of hyperparameters and iteratively adjusts them. It often uses a heuristic approach to select the next set of hyperparameters, based on the results of the previous evaluation.

- Iterative Process: The step-wise method adjusts one hyperparameter at a time while keeping the others fixed. The goal is to reach an optimal or near-optimal set of hyperparameters through a series of steps.

- Complexity: Step-wise methods are typically less computationally intensive than grid search because they explore a smaller portion of the hyperparameter space at each step. This can make them more suitable for cases with limited computational resources.

- Advantages: Step-wise methods can be faster and more efficient than grid search, as they focus on promising regions of the hyperparameter space based on intermediate results. They may also be more suitable for real-time or online learning settings.

- Disadvantages: Step-wise methods might not guarantee finding the global optimum, and they are sensitive to the initial choice of hyperparameters. The final result can be influenced by the order in which hyperparameters are adjusted.

**How does the Stepwise Search Work?**
1. Initialization: The stepwise search starts with an initial set of candidate orders for p, d, and q. These initial orders are typically selected based on common practice and heuristics. For example, it may start with a range of values for p, d, and q (e.g., p=0-2, d=0-2, q=0-2).

2. Stepwise Search: The search algorithm begins by fitting ARIMA models for all combinations of the candidate orders (p, d, q) and evaluates their performance using a chosen criterion, such as AIC (Akaike Information Criterion). It typically considers a few candidate models (not all possible combinations) to speed up the process.

3. Refinement: Based on the initial fits, the algorithm selects the best-fitting model (lowest AIC) among the candidates.

4. Iterative Process: The algorithm iteratively refines the selected model by considering nearby values of p, d, and q. It may explore variations of the current best-fitting model by incrementing or decrementing each order component (e.g., try p+1, p-1, d+1, d-1, q+1, q-1).

5. Stopping Criteria: The stepwise search continues until a predefined stopping criterion is met. The criterion might include a maximum number of iterations or convergence to a model that doesn't significantly improve the chosen criterion (e.g., AIC).

In summary, the stepwise search method is more efficient than trying all possible combinations of p, d, and q because it narrows down the search space to a set of candidate models and iteratively refines the selected model based on performance. This approach helps to find a reasonably good-fitting ARIMA model while avoiding an exhaustive search through all possible combinations, which can be computationally expensive, especially for large datasets.






In [ ]:
# auto_arima function

import pmdarima as pm

def determine_arima_orders(sales_data):
  # Use an automated order selection method (e.g., auto_arima from pmdarima)
  # to determine the best ARIMA orders (p, d, q).
  model = pm.auto_arima(sales_data, seasonal=False, stepwise=True, suppress_warnings=True)
  return model.order

In [ ]:
# function for model fitting

def fit_arima_model(sales_data, orders):
  # Fit the ARIMA model using the determined orders
  p, d, q = orders
  model = ARIMA(sales_data[:-10], order=(p, d, q))  # Use -10 to exclude the last 10 rows for testing
  model_fit = model.fit()  # Use disp=0 to suppress output
  return model_fit

In [ ]:
# function for generating forecasts

def generate_forecast(model_fit, n_forecast):
  # Generate forecasts for a specified number of future periods
  forecast = model_fit.forecast(steps=n_forecast) # , stderr, conf_int
  return forecast

**Forecasting Model Evaluation Methods**

**MAE (Mean Absolute Error):**
- MAE measures the average absolute difference between the forecasted values and the actual values in the test dataset.
- It gives you an idea of the average magnitude of errors in your forecasts.
- Smaller MAE values indicate better accuracy, with zero indicating a perfect forecast.

**MSE (Mean Squared Error):**
- MSE calculates the average of the squared differences between the forecasted values and the actual values in the test dataset.
- Squaring the errors gives more weight to larger errors and less weight to smaller errors.
- Like MAE, smaller MSE values indicate better accuracy, with zero indicating a perfect forecast.

**RMSE (Root Mean Squared Error):**
- RMSE is the square root of the MSE and provides a measure of the typical size of errors in the forecast, in the same units as the original data.
- It gives you an idea of the spread or dispersion of errors.
- Smaller RMSE values indicate better accuracy, with zero indicating a perfect forecast.

**MAPE (Mean Absolute Percentage Error):**
- MAPE calculates the average percentage difference between the forecasted values and the actual values in the test dataset.
- It measures the relative accuracy of the forecasts, which can be useful when you want to understand the forecast's performance in terms of percentages.
- Lower MAPE values indicate better accuracy, with zero indicating a perfect forecast.

**Summary:**
1. MAE and MSE provide measures of the average error magnitude, with MSE giving more weight to larger errors.
2. RMSE gives a similar measure as MSE but in the original units of the data, making it easier to interpret.
3. MAPE provides a measure of the average percentage error, which can be particularly useful when you want to understand the forecast accuracy in relative terms.

In [ ]:
# function for model evaluation methods

def evaluate_forecast(actual, forecast):
  mae = mean_absolute_error(actual, forecast)
  mse = mean_squared_error(actual, forecast)
  rmse = mse ** 0.5

  # Calculate MAPE
  # Handle cases where actual values are 0
  actual = actual.reset_index(drop=True) # resetting the index because actual data has "week" as index
  forecast = forecast.reset_index(drop=True) # resetting the index here too
  nonzero_actual = actual[actual != 0]
  nonzero_forecast = forecast[actual != 0]
  mape = (abs(nonzero_actual - nonzero_forecast) / nonzero_actual).mean() * 100
  '''
  mask = actual != 0
  actual = actual[mask]
  forecast = forecast[mask]
  # Calculate percentage errors
  percentage_errors = np.abs((actual - forecast) / actual) * 100
  # Calculate and return mean percentage error
  mape = np.mean(percentage_errors)
  # mape = (abs(actual - forecast) / actual).mean() * 100
  '''

  return mae, mse, rmse, mape


In [ ]:
# loop over users to fit model, forecast, and evaluate

n_forecast = 10  # Number of weeks to forecast
forecast_results = [] # empty dataframe to store forecast results in

for business in businesses_filtered_10_weeks['business_name'].unique():

  # Filter df on business
  business_df = businesses_filtered_10_weeks.loc[businesses_filtered_10_weeks['business_name'] == business, :]
  business_df = (business_df
                  .loc[:, ['week', 'sales_amount']]
                  .set_index('week')
                  )

  # Extract sales data column
  sales_series = business_df['sales_amount'].astype(float)

  # Determine ARIMA orders (p, d, q) using the previously defined function
  orders = determine_arima_orders(sales_series)

  # Fit ARIMA model and generate forecasts
  model_fit = fit_arima_model(sales_series, orders)
  forecast = generate_forecast(model_fit, n_forecast)

  # Load actual sales data for the forecasted period
  actual_sales = business_df['sales_amount'][-10:].astype(float) # filtering on the last 10 rows

  # Evaluate forecasts
  mae, mse, rmse, mape = evaluate_forecast(actual_sales, forecast)

  # Store results in a dictionary or DataFrame
  result = {
        'business_name': business,
        'arima_orders': orders,
        'forecasts': forecast.tolist(),
        # 'confidence_intervals': conf_int.tolist()
        'actual_sales': actual_sales.tolist(),
        # 'mae': mae,
        # 'mse': mse,
        'rmse': rmse,
        'mape': mape
    }

  # Append the result to the list
  forecast_results.append(result)

# Convert the list of results into a DataFrame (optional)
forecast_df = pd.DataFrame(forecast_results)


In [ ]:
forecast_df

,business_name,arima_orders,forecasts,actual_sales,rmse,mape
0,4B Foods - RDS,"(1, 0, 0)","[32935.233939388185, 40489.29223759308, 45457....","[11015.0, 54180.0, 104650.0, 140637.0, 99988.0...",4.503401e+04,240.205727
1,AADC RDS,"(1, 0, 1)","[289734.79332501505, 524191.4395953347, 643348...","[206072.0, 474450.0, 553031.0, 501251.0, 53143...",2.243669e+05,44.847864
2,Abbasi Trader's,"(1, 1, 0)","[534658.9664844028, 616545.2814183125, 584279....","[207163.0, 922020.0, 975692.0, 877991.0, 59804...",3.155557e+05,101.012290
3,Abubakar Traders,"(0, 0, 0)","[166755.62500000006, 166755.62500000006, 16675...","[146395.0, 288477.0, 131585.0, 208760.0, 69425...",1.181704e+05,779.618393
4,Afshar Enterprises RDS,"(0, 1, 0)","[434150.0, 434150.0, 434150.0, 434150.0, 43415...","[139250.0, 511300.0, 550890.0, 488940.0, 12483...",7.085555e+05,63.765362
5,Anaaj Traders,"(1, 0, 0)","[591060.5097494442, 519576.17444904195, 468400...","[712388.0, 1005234.0, 775958.0, 214395.0, 2791...",2.647688e+05,293.288917
6,FM Foods,"(0, 0, 0)","[26024.555396853204, 26024.555396853204, 26024...","[11724.0, 29915.0, 26294.0, 53766.0, 41886.0, ...",1.202914e+04,35.911358
7,Faisal Traders - RDS,"(5, 1, 3)","[95734.74822521606, 152915.3290443821, 91112.8...","[325629.0, 382804.0, 154782.0, 226085.0, 14704...",2.645117e+05,54.633992
8,Fariz Traders,"(3, 0, 0)","[359973.5141734303, 343002.91303602414, 328228...","[364903.0, 335251.0, 211329.0, 146973.0, 14940...",1.128435e+05,62.364577
9,Good Luck,"(2, 0, 0)","[1695859.6485624826, 1726732.1661636264, 17124...","[656734.0, 918476.0, 289433.0, 293931.0, 9275....",1.524308e+06,52931.559212


In [ ]:
print("Mean MAPE Score:", forecast_df['mape'].mean())
print("Median MAPE Score:", forecast_df['mape'].median())
print("Percentage of Users with MAPE < 50:", round(round(forecast_df.loc[forecast_df['mape'] < 50, :].shape[0] / forecast_df.shape[0], 2)*100), "%")

Mean MAPE Score: 2850.442722952407
Median MAPE Score: 85.48855033716858
Percentage of Users with MAPE < 50: 23 %


### **Testing SARIMA**

In [ ]:
# auto_arima function
import pmdarima as pm
def determine_arima_orders(sales_data):
  # Use an automated order selection method (e.g., auto_arima from pmdarima)
  # to determine the best ARIMA orders (p, d, q).
  model = pm.auto_arima(sales_data, seasonal=True, stepwise=True, suppress_warnings=True)
  return model.order, model.seasonal_order

# function for model fitting
def fit_arima_model(sales_data, orders, seasonal_orders):
  # Fit the SARIMA model using the determined orders
  p, d, q = orders
  P, D, Q, s = seasonal_orders  # Specify seasonal order parameters
  model = SARIMAX(sales_data[:-10], order=(p, d, q), seasonal_order=(P, D, Q, s))  # Use -10 to exclude the last 10 rows for testing
  model_fit = model.fit(disp=0)  # Use disp=0 to suppress output
  return model_fit

# function for generating forecasts -- same as before
# function for model evaluation methods -- same as before

In [ ]:
# loop over users to fit model, forecast, and evaluate

n_forecast = 10  # Number of weeks to forecast
forecast_results = [] # empty dataframe to store forecast results in

for business in businesses_filtered_10_weeks['business_name'].unique():

  # Filter df on business
  business_df = businesses_filtered_10_weeks.loc[businesses_filtered_10_weeks['business_name'] == business, :]
  business_df = (business_df
                  .loc[:, ['week', 'sales_amount']]
                  .set_index('week')
                  )

  # Extract sales data column
  sales_series = business_df['sales_amount'].astype(float)

  # Determine SARIMA orders (p, d, q) and seasonal orders (P, D, Q, S)
  orders, seasonal_orders = determine_arima_orders(sales_series)

  # Fit ARIMA model and generate forecasts
  model_fit = fit_arima_model(sales_series, orders, seasonal_orders)
  forecast = generate_forecast(model_fit, n_forecast)

  # Load actual sales data for the forecasted period
  actual_sales = business_df['sales_amount'][-10:].astype(float) # filtering on the last 10 rows

  # Evaluate forecasts
  mae, mse, rmse, mape = evaluate_forecast(actual_sales, forecast)

  # Store results in a dictionary or DataFrame
  result = {
        'business_name': business,
        'arima_orders': orders,
        'seasonal_arima_orders': seasonal_orders,
        'forecasts': forecast.tolist(),
        # 'confidence_intervals': conf_int.tolist()
        'actual_sales': actual_sales.tolist(),
        # 'mae': mae,
        # 'mse': mse,
        'rmse': rmse,
        'mape': mape
    }

  # Append the result to the list
  forecast_results.append(result)

# Convert the list of results into a DataFrame (optional)
forecast_df = pd.DataFrame(forecast_results)


In [ ]:
forecast_df

,business_name,arima_orders,seasonal_arima_orders,forecasts,actual_sales,rmse,mape
0,4B Foods - RDS,"(1, 0, 0)","(0, 0, 0, 0)","[18249.2252712223, 15526.07100232255, 13209.26...","[11015.0, 54180.0, 104650.0, 140637.0, 99988.0...",7.444407e+04,82.862568
1,AADC RDS,"(1, 0, 1)","(0, 0, 0, 0)","[102321.76346276917, 84087.94833593625, 69103....","[206072.0, 474450.0, 553031.0, 501251.0, 53143...",4.564674e+05,87.231492
2,Abbasi Trader's,"(1, 1, 0)","(0, 0, 0, 0)","[534658.9664844028, 616545.2814183125, 584279....","[207163.0, 922020.0, 975692.0, 877991.0, 59804...",3.155557e+05,101.012290
3,Abubakar Traders,"(0, 0, 0)","(0, 0, 0, 0)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[146395.0, 288477.0, 131585.0, 208760.0, 69425...",1.314201e+05,100.000000
4,Afshar Enterprises RDS,"(0, 1, 0)","(0, 0, 0, 0)","[434150.0, 434150.0, 434150.0, 434150.0, 43415...","[139250.0, 511300.0, 550890.0, 488940.0, 12483...",7.085555e+05,63.765362
5,Anaaj Traders,"(1, 0, 0)","(0, 0, 0, 0)","[638064.92436261, 589259.2087598051, 544186.65...","[712388.0, 1005234.0, 775958.0, 214395.0, 2791...",2.532429e+05,278.274492
6,FM Foods,"(0, 0, 0)","(0, 0, 0, 0)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[11724.0, 29915.0, 26294.0, 53766.0, 41886.0, ...",3.141111e+04,100.000000
7,Faisal Traders - RDS,"(5, 1, 3)","(0, 0, 0, 0)","[95734.74822521606, 152915.3290443821, 91112.8...","[325629.0, 382804.0, 154782.0, 226085.0, 14704...",2.645117e+05,54.633992
8,Fariz Traders,"(0, 0, 2)","(0, 0, 0, 0)","[277459.81725786976, 99081.6805361805, 0.0, 0....","[364903.0, 335251.0, 211329.0, 146973.0, 14940...",2.497584e+05,89.440893
9,Good Luck,"(2, 0, 0)","(0, 0, 0, 0)","[1547903.113397108, 1512885.6273481876, 140172...","[656734.0, 918476.0, 289433.0, 293931.0, 9275....",1.022081e+06,33046.503678


Possible reasons for getting (0, 0, 0, 0) as the optimal seasonal order:

- The data may not exhibit a strong seasonal pattern, and SARIMA may not be the most appropriate model for forecasting in this case.

- The data may have been preprocessed or transformed in a way that removed or reduced seasonality, making it less prominent.

- The auto_arima function's algorithm may not have been able to detect the seasonality due to the specific characteristics of the data.




In [ ]:
print("Mean MAPE Score:", forecast_df['mape'].mean())
print("Median MAPE Score:", forecast_df['mape'].median())
print("Percentage of Users with MAPE < 50:", round(round(forecast_df.loc[forecast_df['mape'] < 50, :].shape[0] / forecast_df.shape[0], 2)*100), "%")

Mean MAPE Score: 1733.5587985473853
Median MAPE Score: 90.35156076796595
Percentage of Users with MAPE < 50: 19 %


# **Parameter Tuning**

## **Overall**

In [ ]:
# auto_arima function
import pmdarima as pm
def determine_arima_orders(sales_data):
  # Use an automated order selection method (e.g., auto_arima from pmdarima)
  # to determine the best ARIMA orders (p, d, q).
  model = pm.auto_arima(sales_data, seasonal=False, stepwise=True, suppress_warnings=True, information_criterion='bic', trend='n', start_p=0, start_q=0, max_p=10, max_q=10)
  return model.order

# function for model fitting
def fit_arima_model(sales_data, orders):
  # Fit the ARIMA model using the determined orders
  p, d, q = orders
  model = ARIMA(sales_data[:-10], order=(p, d, q))  # Use -10 to exclude the last 10 rows for testing
  model_fit = model.fit()  # Use disp=0 to suppress output
  return model_fit

# function for generating forecasts
def generate_forecast(model_fit, n_forecast):
  # Generate forecasts for a specified number of future periods
  forecast = model_fit.forecast(steps=n_forecast) # , stderr, conf_int
  return forecast

# function for model evaluation methods
def evaluate_forecast(actual, forecast):
  mae = mean_absolute_error(actual, forecast)
  mse = mean_squared_error(actual, forecast)
  rmse = mse ** 0.5
  # Calculate MAPE
  # Handle cases where actual values are 0
  actual = actual.reset_index(drop=True) # resetting the index because actual data has "week" as index
  forecast = forecast.reset_index(drop=True) # resetting the index here too
  nonzero_actual = actual[actual != 0]
  nonzero_forecast = forecast[actual != 0]
  mape = (abs(nonzero_actual - nonzero_forecast) / nonzero_actual).mean() * 100
  '''
  mask = actual != 0
  actual = actual[mask]
  forecast = forecast[mask]
  # Calculate percentage errors
  percentage_errors = np.abs((actual - forecast) / actual) * 100
  # Calculate and return mean percentage error
  mape = np.mean(percentage_errors)
  # mape = (abs(actual - forecast) / actual).mean() * 100
  '''
  return mae, mse, rmse, mape





# loop over users to fit model, forecast, and evaluate

n_forecast = 10  # Number of weeks to forecast
forecast_results = [] # empty dataframe to store forecast results in

for business in businesses_filtered_10_weeks['business_name'].unique():

  # Filter df on business
  business_df = businesses_filtered_10_weeks.loc[businesses_filtered_10_weeks['business_name'] == business, :]
  business_df = (business_df
                  .loc[:, ['week', 'sales_amount']]
                  .set_index('week')
                  )

  # Extract sales data column
  sales_series = business_df['sales_amount'].astype(float)

  # Determine ARIMA orders (p, d, q) using the previously defined function
  orders = determine_arima_orders(sales_series)

  # Fit ARIMA model and generate forecasts
  model_fit = fit_arima_model(sales_series, orders)
  forecast = generate_forecast(model_fit, n_forecast)

  # Load actual sales data for the forecasted period
  actual_sales = business_df['sales_amount'][-10:].astype(float) # filtering on the last 10 rows

  # Evaluate forecasts
  mae, mse, rmse, mape = evaluate_forecast(actual_sales, forecast)

  # Store results in a dictionary or DataFrame
  result = {
        'business_name': business,
        'arima_orders': orders,
        'forecasts': forecast.tolist(),
        # 'confidence_intervals': conf_int.tolist()
        'actual_sales': actual_sales.tolist(),
        # 'mae': mae,
        # 'mse': mse,
        'rmse': rmse,
        'mape': mape
    }

  # Append the result to the list
  forecast_results.append(result)

# Convert the list of results into a DataFrame (optional)
forecast_df = pd.DataFrame(forecast_results)



/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/p

In [ ]:
forecast_df

,business_name,arima_orders,forecasts,actual_sales,rmse,mape
0,4B Foods - RDS,"(1, 0, 0)","[32935.233939388185, 40489.29223759308, 45457....","[11015.0, 54180.0, 104650.0, 140637.0, 99988.0...",4.503401e+04,240.205727
1,AADC RDS,"(1, 0, 0)","[400271.958621896, 505095.99595555675, 579916....","[206072.0, 474450.0, 553031.0, 501251.0, 53143...",2.021040e+05,42.771760
2,Abbasi Trader's,"(1, 1, 0)","[534658.9664844028, 616545.2814183125, 584279....","[207163.0, 922020.0, 975692.0, 877991.0, 59804...",3.155557e+05,101.012290
3,Abubakar Traders,"(1, 0, 1)","[176529.51925154016, 171277.83097874493, 16884...","[146395.0, 288477.0, 131585.0, 208760.0, 69425...",1.180333e+05,780.496243
4,Afshar Enterprises RDS,"(0, 1, 0)","[434150.0, 434150.0, 434150.0, 434150.0, 43415...","[139250.0, 511300.0, 550890.0, 488940.0, 12483...",7.085555e+05,63.765362
5,Anaaj Traders,"(1, 0, 0)","[591060.5097494442, 519576.17444904195, 468400...","[712388.0, 1005234.0, 775958.0, 214395.0, 2791...",2.647688e+05,293.288917
6,FM Foods,"(2, 0, 1)","[27540.936220133375, 25038.38875078393, 26656....","[11724.0, 29915.0, 26294.0, 53766.0, 41886.0, ...",1.230956e+04,37.528678
7,Faisal Traders - RDS,"(0, 1, 2)","[121942.42794031616, 129563.40905584766, 12956...","[325629.0, 382804.0, 154782.0, 226085.0, 14704...",2.541550e+05,46.483875
8,Fariz Traders,"(1, 0, 0)","[352145.43618770776, 341879.84377057134, 33271...","[364903.0, 335251.0, 211329.0, 146973.0, 14940...",1.166639e+05,65.283700
9,Good Luck,"(2, 0, 0)","[1695859.6485624826, 1726732.1661636264, 17124...","[656734.0, 918476.0, 289433.0, 293931.0, 9275....",1.524308e+06,52931.559212


In [ ]:
# pm.auto_arima(sales_data, seasonal=False, stepwise=True, suppress_warnings=True)
print("Mean MAPE Score:", forecast_df['mape'].mean())
print("Median MAPE Score:", forecast_df['mape'].median())

Mean MAPE Score: 2850.376419471664
Median MAPE Score: 85.48855033716858


In [ ]:
# pm.auto_arima(sales_data, seasonal=True, m=1, stepwise=True, suppress_warnings=True)
print("Mean MAPE Score:", forecast_df['mape'].mean())
print("Median MAPE Score:", forecast_df['mape'].median())

Mean MAPE Score: 2845.2261783157123
Median MAPE Score: 83.93057274368684


In [ ]:
# pm.auto_arima(sales_data, seasonal=True, m=2, stepwise=True, suppress_warnings=True)
print("Mean MAPE Score:", forecast_df['mape'].mean())
print("Median MAPE Score:", forecast_df['mape'].median())

Mean MAPE Score: 2897.2407945330337
Median MAPE Score: 96.1384091632307


In [ ]:
# pm.auto_arima(sales_data, seasonal=True, stepwise=True, suppress_warnings=True)
print("Mean MAPE Score:", forecast_df['mape'].mean())
print("Median MAPE Score:", forecast_df['mape'].median())

Mean MAPE Score: 2850.4078352464367
Median MAPE Score: 89.08967995621114


In [ ]:
# pm.auto_arima(sales_data, seasonal=False, stepwise=True, suppress_warnings=True, information_criterion='bic')
print("Mean MAPE Score:", forecast_df['mape'].mean())
print("Median MAPE Score:", forecast_df['mape'].median())

# information_criterion='bic' resulting in lower MAPE score

Mean MAPE Score: 2846.648676611885
Median MAPE Score: 83.93057274368684


In [ ]:
# pm.auto_arima(sales_data, seasonal=False, stepwise=True, suppress_warnings=True, information_criterion='bic', trend='t')
print("Mean MAPE Score:", forecast_df['mape'].mean())
print("Median MAPE Score:", forecast_df['mape'].median())

# trend='t' with the lowest MAPE score amongst all values

Mean MAPE Score: 2844.8910658367113
Median MAPE Score: 83.93057274368684


In [ ]:
# pm.auto_arima(sales_data, seasonal=False, stepwise=False, suppress_warnings=True, information_criterion='bic', trend='t')
print("Mean MAPE Score:", forecast_df['mape'].mean())
print("Median MAPE Score:", forecast_df['mape'].median())

# both step-wise and grid search methods resulting in same MAPE score

Mean MAPE Score: 2829.2060740401625
Median MAPE Score: 83.93057274368684


In [ ]:
# pm.auto_arima(sales_data, seasonal=False, stepwise=True, suppress_warnings=True, information_criterion='bic', trend='n', start_p=0, start_q=0, max_p=10, max_q=10)
print("Mean MAPE Score:", forecast_df['mape'].mean())
print("Median MAPE Score:", forecast_df['mape'].median())

Mean MAPE Score: 2856.056288700776
Median MAPE Score: 83.93057274368684


In [ ]:
print("Mean MAPE Score:", forecast_df['mape'].mean())
print("Median MAPE Score:", forecast_df['mape'].median())

In [ ]:
print("Mean MAPE Score:", forecast_df['mape'].mean())
print("Median MAPE Score:", forecast_df['mape'].median())

In [ ]:
print("Mean MAPE Score:", forecast_df['mape'].mean())
print("Median MAPE Score:", forecast_df['mape'].median())

## **information_criterion ('aic', 'bic', 'hqic', 'oob')**

- **AIC (Akaike Information Criterion)**: AIC is a measure of the model's goodness of fit while penalizing for model complexity. It seeks to balance model accuracy and simplicity.
- **BIC (Bayesian Information Criterion)**: BIC, similar to AIC, evaluates model fit and complexity. It is stricter in penalizing complex models compared to AIC.
- **HQIC (Hannan-Quinn Information Criterion)**: HQIC also balances model fit and complexity but provides a more substantial penalty for complexity than AIC and less than BIC (HQIC is suitable when you want a balance between AIC and BIC).
- **Out-of-Bag (OOB) Error (for Random Forest and Ensemble Models)**: OOB error is specific to ensemble methods like Random Forest, where data is split into in-bag (used for training) and out-of-bag (used for validation) samples. OOB error is a measure of model performance on the out-of-bag samples.

In [ ]:
comparison_results = []

for information_criterion_value in ('aic', 'bic', 'hqic', 'oob'):

  # auto_arima function
  import pmdarima as pm
  def determine_arima_orders(sales_data):
    # Use an automated order selection method (e.g., auto_arima from pmdarima)
    # to determine the best ARIMA orders (p, d, q).
    model = pm.auto_arima(sales_data, seasonal=False, stepwise=True, suppress_warnings=True, information_criterion=information_criterion_value, trend='t')
    return model.order

  #------------------
  # loop over users to fit model, forecast, and evaluate

  n_forecast = 10  # Number of weeks to forecast
  forecast_results = [] # empty dataframe to store forecast results in

  for business in businesses_filtered_10_weeks['business_name'].unique():

    # Filter df on business
    business_df = businesses_filtered_10_weeks.loc[businesses_filtered_10_weeks['business_name'] == business, :]
    business_df = (business_df
                    .loc[:, ['week', 'sales_amount']]
                    .set_index('week')
                    )

    # Extract sales data column
    sales_series = business_df['sales_amount'].astype(float)

    # Determine ARIMA orders (p, d, q) using the previously defined function
    orders = determine_arima_orders(sales_series)

    # Fit ARIMA model and generate forecasts
    model_fit = fit_arima_model(sales_series, orders)
    forecast = generate_forecast(model_fit, n_forecast)

    # Load actual sales data for the forecasted period
    actual_sales = business_df['sales_amount'][-10:].astype(float) # filtering on the last 10 rows

    # Evaluate forecasts
    mae, mse, rmse, mape = evaluate_forecast(actual_sales, forecast)

    # Store results in a dictionary or DataFrame
    result = {
          'business_name': business,
          'arima_orders': orders,
          'forecasts': forecast.tolist(),
          # 'confidence_intervals': conf_int.tolist()
          'actual_sales': actual_sales.tolist(),
          # 'mae': mae,
          # 'mse': mse,
          'rmse': rmse,
          'mape': mape
      }

    # Append the result to the list
    forecast_results.append(result)

  # Convert the list of results into a DataFrame (optional)
  forecast_df = pd.DataFrame(forecast_results)

  comparison = {
      'information_criterion': information_criterion_value,
      'median_mape_score': forecast_df['mape'].median()
  }
  comparison_results.append(comparison)

comparison_df = pd.DataFrame(comparison_results)

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/p

In [ ]:
comparison_df

,information_criterion,median_mape_score
0,aic,85.488550
1,bic,83.930573
2,hqic,85.488550
3,oob,85.488550


## **trend ('n', 'c', 't', 'ct')**

- "n": No trend (stationary series) -- An example of time series with no trend includes noise data, where observations exhibit random fluctuations without any underlying trend or seasonality.
- "c": Constant trend (default) -- An example of a constant trend could be monthly sales data for a retail store that steadily increases over time due to consistent business growth.
- "t": Linear trend -- An example of a linear trend could be a time series of a company's stock price, which exhibits a gradual increase or decrease in value over time.
- "ct": Constant and linear trend (i.e., both a constant and a linear component) -- An example of a "ct" trend could be a time series of a city's population, where the population steadily increases over time due to consistent growth (linear trend) but also experiences constant immigration or births (constant shift) over the years.

In [ ]:
comparison_results = []

for trend_value in ('n', 'c', 't', 'ct'):

  # auto_arima function
  import pmdarima as pm
  def determine_arima_orders(sales_data):
    # Use an automated order selection method (e.g., auto_arima from pmdarima)
    # to determine the best ARIMA orders (p, d, q).
    model = pm.auto_arima(sales_data, seasonal=False, stepwise=True, suppress_warnings=True, information_criterion='aic', trend=trend_value)
    return model.order

  #------------------
  # loop over users to fit model, forecast, and evaluate

  n_forecast = 10  # Number of weeks to forecast
  forecast_results = [] # empty dataframe to store forecast results in

  for business in businesses_filtered_10_weeks['business_name'].unique():

    # Filter df on business
    business_df = businesses_filtered_10_weeks.loc[businesses_filtered_10_weeks['business_name'] == business, :]
    business_df = (business_df
                    .loc[:, ['week', 'sales_amount']]
                    .set_index('week')
                    )

    # Extract sales data column
    sales_series = business_df['sales_amount'].astype(float)

    # Determine ARIMA orders (p, d, q) using the previously defined function
    orders = determine_arima_orders(sales_series)

    # Fit ARIMA model and generate forecasts
    model_fit = fit_arima_model(sales_series, orders)
    forecast = generate_forecast(model_fit, n_forecast)

    # Load actual sales data for the forecasted period
    actual_sales = business_df['sales_amount'][-10:].astype(float) # filtering on the last 10 rows

    # Evaluate forecasts
    mae, mse, rmse, mape = evaluate_forecast(actual_sales, forecast)

    # Store results in a dictionary or DataFrame
    result = {
          'business_name': business,
          'arima_orders': orders,
          'forecasts': forecast.tolist(),
          # 'confidence_intervals': conf_int.tolist()
          'actual_sales': actual_sales.tolist(),
          # 'mae': mae,
          # 'mse': mse,
          'rmse': rmse,
          'mape': mape
      }

    # Append the result to the list
    forecast_results.append(result)

  # Convert the list of results into a DataFrame (optional)
  forecast_df = pd.DataFrame(forecast_results)

  comparison = {
      'trend': trend_value,
      'median_mape_score': forecast_df['mape'].median()
  }
  comparison_results.append(comparison)

comparison_df = pd.DataFrame(comparison_results)

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/p

In [ ]:
comparison_df

,trend,median_mape_score
0,n,83.930573
1,c,89.089680
2,t,85.488550
3,ct,89.089680


## **stepwise (True, False)**

There are 2 methods for selecting model hyperparameters and optimizing machine learning or time series models. They differ in their search strategies and complexity:

**Grid Search:**
- Search Strategy: In a grid search, you specify a range of possible values for each hyperparameter, and the search method evaluates every possible combination of hyperparameters within those specified ranges. It creates a grid of all possible combinations and systematically tests each one.

- Exhaustive Search: Grid search is exhaustive, meaning it doesn't skip any combination. It tests all possibilities, making it a comprehensive approach.

- Complexity: Grid search can be computationally expensive, especially when dealing with a large number of hyperparameters or a wide range of values for each hyperparameter. The search space grows exponentially with the number of hyperparameters.

- Advantages: Grid search is guaranteed to find the best combination of hyperparameters within the specified ranges, given enough computational resources and time. It is a simple and straightforward approach.

- Disadvantages: It can be slow and computationally intensive, especially when dealing with a large search space. It may not be feasible in situations with limited computational resources.


**Step-Wise Search:**

- Search Strategy: A step-wise search, also known as a sequential search, starts with an initial set of hyperparameters and iteratively adjusts them. It often uses a heuristic approach to select the next set of hyperparameters, based on the results of the previous evaluation.

- Iterative Process: The step-wise method adjusts one hyperparameter at a time while keeping the others fixed. The goal is to reach an optimal or near-optimal set of hyperparameters through a series of steps.

- Complexity: Step-wise methods are typically less computationally intensive than grid search because they explore a smaller portion of the hyperparameter space at each step. This can make them more suitable for cases with limited computational resources.

- Advantages: Step-wise methods can be faster and more efficient than grid search, as they focus on promising regions of the hyperparameter space based on intermediate results. They may also be more suitable for real-time or online learning settings.

- Disadvantages: Step-wise methods might not guarantee finding the global optimum, and they are sensitive to the initial choice of hyperparameters. The final result can be influenced by the order in which hyperparameters are adjusted.

In [ ]:
comparison_results = []

for stepwise_value in (True, False):

  # auto_arima function
  import pmdarima as pm
  def determine_arima_orders(sales_data):
    # Use an automated order selection method (e.g., auto_arima from pmdarima)
    # to determine the best ARIMA orders (p, d, q).
    model = pm.auto_arima(sales_data, seasonal=False, stepwise=stepwise_value, suppress_warnings=True, information_criterion='aic', trend='t')
    return model.order

  #------------------
  # loop over users to fit model, forecast, and evaluate

  n_forecast = 10  # Number of weeks to forecast
  forecast_results = [] # empty dataframe to store forecast results in

  for business in businesses_filtered_10_weeks['business_name'].unique():

    # Filter df on business
    business_df = businesses_filtered_10_weeks.loc[businesses_filtered_10_weeks['business_name'] == business, :]
    business_df = (business_df
                    .loc[:, ['week', 'sales_amount']]
                    .set_index('week')
                    )

    # Extract sales data column
    sales_series = business_df['sales_amount'].astype(float)

    # Determine ARIMA orders (p, d, q) using the previously defined function
    orders = determine_arima_orders(sales_series)

    # Fit ARIMA model and generate forecasts
    model_fit = fit_arima_model(sales_series, orders)
    forecast = generate_forecast(model_fit, n_forecast)

    # Load actual sales data for the forecasted period
    actual_sales = business_df['sales_amount'][-10:].astype(float) # filtering on the last 10 rows

    # Evaluate forecasts
    mae, mse, rmse, mape = evaluate_forecast(actual_sales, forecast)

    # Store results in a dictionary or DataFrame
    result = {
          'business_name': business,
          'arima_orders': orders,
          'forecasts': forecast.tolist(),
          # 'confidence_intervals': conf_int.tolist()
          'actual_sales': actual_sales.tolist(),
          # 'mae': mae,
          # 'mse': mse,
          'rmse': rmse,
          'mape': mape
      }

    # Append the result to the list
    forecast_results.append(result)

  # Convert the list of results into a DataFrame (optional)
  forecast_df = pd.DataFrame(forecast_results)

  comparison = {
      'stepwise': "Grid Search" if stepwise_value==False else "Step-wise Search",
      'median_mape_score': forecast_df['mape'].median()
  }
  comparison_results.append(comparison)

comparison_df = pd.DataFrame(comparison_results)

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/p

In [ ]:
comparison_df

,stepwise,median_mape_score
0,Step-wise Search,85.48855
1,Grid Search,85.48855


# **Forecasting on Original Non-transformed series**

## **Defining Model Parameter Combinations**

In [ ]:
import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import pandas as pd
import statsmodels.api as sm
import matplotlib
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

## **Functions for Calculating Errors**

for each business :
*   get the best hyperparameter for ARIMA MODEL
*   and save it in dataframe called "data"



In [ ]:
import numpy as np

def mean_absolute_percentage_error(y_true, y_pred):
    """
    Calculate Mean Absolute Percentage Error (MAPE).

    Parameters:
    - y_true: Array of true values.
    - y_pred: Array of predicted values.

    Returns:
    - MAPE value.
    """
    # Handle cases where y_true contains zero values
    mask = y_true != 0
    y_true = y_true[mask]
    y_pred = y_pred[mask]

    # Calculate percentage errors
    percentage_errors = np.abs((y_true - y_pred) / y_true) * 100

    # Calculate and return mean percentage error
    mape = np.mean(percentage_errors)
    return mape
def root_mean_square_error(y_true, y_pred):
    """
    Calculate Root Mean Square Error (RMSE).

    Parameters:
    - y_true: Array of true values.
    - y_pred: Array of predicted values.

    Returns:
    - RMSE value.
    """
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    return rmse


## **Applying the Model**

In [ ]:
data = businesses_filtered_10_weeks

In [ ]:
# data_=data[data['partner']=='Partner X sellers']
data_ = data

for business_name in data_['business_name'].unique()[:]: #using only 5 business now for time; but this will done for all business
  # data_ = data[data['partner']=='Partner X sellers']
  data_ = data
  data_ = data_[data_['business_name'] == business_name].groupby('week')['sales_amount'].sum().reset_index().sort_values(by='week')
  if data_.shape[0] > 10 : #only business with 10 weeks of data will be participating in time series
    data_test = data_[-5:] #last 5 week will be used in testing
    data_ = data_[:-5]
    print("business_name", business_name)
    temp = list(data_['sales_amount'].values)
    data_
    print("number of records",len(temp))
    smallest_aic = 1000000000
    for param in pdq:

        for param_seasonal in seasonal_pdq:

            try:

                mod = sm.tsa.statespace.SARIMAX(temp,
                                                order=param,
                                                seasonal_order=param_seasonal,
                                                enforce_stationarity=False,
                                                enforce_invertibility=False)
                results = mod.fit()
                forecast_steps = 5  # Number of steps to forecast
                forecast = results.forecast(steps=forecast_steps)
                predicted_values = np.array(forecast)

                if results.aic < smallest_aic and 0 not in predicted_values :
                  #output = f'ARIMA{param}x{param_seasonal}12 - AIC:{results.aic}')
                  smallest_aic = results.aic
                  data.loc[data['business_name']==business_name,'param'] = str(param)
                  data.loc[data['business_name']==business_name,'param_seasonal'] = str(param_seasonal)
                  data.loc[data['business_name']==business_name,'aic'] = str(smallest_aic)


            except Exception as e:
                print("error:",e)

    #using the best hyper parameter for current business_name

    print(f"smallest AIC {smallest_aic} for {business_name}")
    mod = sm.tsa.statespace.SARIMAX(temp,
                                      order=tuple(int(x) for x in data[data['business_name']==business_name]['param'].values[0].strip('()').split(',')), #take from above df
                                      seasonal_order=tuple(int(x) for x in data[data['business_name']==business_name]['param_seasonal'].values[0].strip('()').split(',')), #take from above df
                                      enforce_stationarity=False,
                                      enforce_invertibility=False)
    mod = mod.fit()
    print(mod.summary())

    # Forecast using the ARIMA model
    forecast_steps = 5  # Number of steps to forecast
    forecast = mod.forecast(steps=forecast_steps)


    print(f"next {forecast_steps} forecasted prediction based on training data is ",forecast)

    # Sample data (replace with your actual data)
    true_values = np.array(data_test['sales_amount'])
    predicted_values = np.array(forecast)

    # Calculate MAPE
    mape = mean_absolute_percentage_error(true_values, predicted_values)

    # Calculate RMSE
    rmse = root_mean_square_error(true_values, predicted_values)

    print("Mean Absolute Percentage Error (MAPE):", mape)
    print("Root Mean Square Error (RMSE):", rmse , )
    print(f"An RMSE value of {rmse} indicates that, on average, our predictions differ from the actual sum_amount values by approximately {rmse} units ")
    data.loc[data['business_name']==business_name,'mape_score'] = str(round(mape))
    data.loc[data['business_name']==business_name,'rmse_score'] = str(round(rmse))
    data.loc[data['business_name']==business_name,'actual_test_records'] = str(true_values)
    data.loc[data['business_name']==business_name,'predicted_records'] = str(predicted_values)

    print(f"smallest AIC {smallest_aic} for {business_name}")

  else:
    print(f"business name {business_name} has less than 10 weeks of training data")



In [ ]:

scores_2 = data[data['param'].notnull()][['business_name', 'param', 'param_seasonal', 'aic','mape_score','rmse_score','actual_test_records','predicted_records']].drop_duplicates()

# data[data['param'].notnull()][['business_name', 'param', 'param_seasonal', 'aic','mape_score','rmse_score','actual_test_records','predicted_records']].drop_duplicates().to_csv("partnership_forecasting_results.csv")
# 'partner',

In [ ]:
scores_2.shape

In [ ]:
scores_2

In [ ]:
# business_name	arima_orders	forecasts	actual_sales	mae	mse	rmse	mape
scores_2 = (scores_2
            .rename(columns={'param':'param_old', 'rmse_score':'rmse_old', 'mape_score':'mape_old'})
            .loc[:, ['business_name', 'param_old', 'rmse_old', 'mape_old']]
            )

# **Combining Scores from Both Approaches**

In [ ]:
scores_final = pd.merge(forecast_df, scores_2, how = "left", on = "business_name")
scores_final = (scores_final
                .loc[:, ['business_name', 'arima_orders', 'param_old', 'rmse', 'rmse_old', 'mape', 'mape_old']]
                .assign(rmse = scores_final['rmse'].apply(lambda x: int(x)),
                        mape = scores_final['mape'].apply(lambda x: int(x)),
                        rmse_old = scores_final['rmse_old'].apply(lambda x: int(x)),
                        mape_old = scores_final['mape_old'].apply(lambda x: int(x))
                        )
                )
scores_final

In [ ]:
scores_final = scores_final.loc[~scores_final['business_name'].isin(['Rupal Trader RDS']), :] # ['rmse', 'rmse_old']
scores_final

In [ ]:
# mean
scores_final[['rmse', 'rmse_old', 'mape', 'mape_old']].mean()

In [ ]:
# median
scores_final[['rmse', 'rmse_old', 'mape', 'mape_old']].median()

In [ ]:
print("Rows with new method generating lower MAPE score: ", round(scores_final.loc[scores_final['mape'] < scores_final['mape_old']].shape[0] / scores_final.shape[0] * 100), "%")
print("Rows with new method generating lower RMSE score: ", round(scores_final.loc[scores_final['rmse'] < scores_final['rmse_old']].shape[0] / scores_final.shape[0] * 100), "%")